In [1]:
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score, train_test_split

# Data Path

In [2]:
data1 = "/Users/Derrick/Desktop/data science new topic/SAT_Results.csv" # data path to data
data2 = "/Users/Derrick/Desktop/data science new topic/nysd.csv"
data3 = "/Users/Derrick/Desktop/data science new topic/NYS_Math_Test_Results_By_Grade_2006-2011_-_School_Level_-_All_Students.csv"
data4 = "/Users/Derrick/Desktop/data science new topic/School_Attendance_and_Enrollment_Statistics_by_District__2010-11_.csv"
data5 = "/Users/Derrick/Desktop/data science new topic/School_Demographics_and_Accountability_Snapshot_2006-2012.csv"
data6 = "/Users/Derrick/Desktop/data science new topic/Graduation_Outcomes_-_Classes_Of_2005-2010_-_School_Level.csv"
data7 = "/Users/Derrick/Desktop/data science new topic/AP__College_Board__2010_School_Level_Results.csv"
data8 = "/Users/Derrick/Desktop/data science new topic/2010-2011_Class_Size_-_School-level_detail.csv"
data9 = "/Users/Derrick/Desktop/data science new topic/masterfile11_d75_final.xlsx"
data10 = "/Users/Derrick/Desktop/data science new topic/masterfile11_gened_final.xlsx"
data11 = "/Users/Derrick/Desktop/data science new topic/Survey Data Dictionary.xls"
data12 = "/Users/Derrick/Desktop/data science new topic/DOE_High_School_Directory_2014-2015.csv"
data13 = "/Users/Derrick/Desktop/data science new topic/School_Progress_Report_2010-2011.csv"
data14 = "/Users/Derrick/Desktop/data science new topic/School_District_Breakdowns.csv"
data15 = "/Users/Derrick/Desktop/data science new topic/attendance.csv"

# Cleaning Data

### SAT result data
#### In this data set, it only provided the scores in 3 categories, and we need to add a aveage total score column 
#### and we only need two column out of it, DBN and Total Average Sco

In [4]:
sat_result = pd.read_csv(data1)
#sat_result.head()

In [5]:
sat_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 6 columns):
DBN                                478 non-null object
SCHOOL NAME                        478 non-null object
Num of SAT Test Takers             478 non-null object
SAT Critical Reading Avg. Score    478 non-null object
SAT Math Avg. Score                478 non-null object
SAT Writing Avg. Score             478 non-null object
dtypes: object(6)
memory usage: 22.5+ KB


In [7]:
new_sat_result = sat_result.drop("Num of SAT Test Takers", 1)
new_sat_result['SAT Math Avg. Score'] = new_sat_result['SAT Math Avg. Score'].convert_objects(convert_numeric = True)
new_sat_result['SAT Critical Reading Avg. Score'] = new_sat_result['SAT Critical Reading Avg. Score'].convert_objects(convert_numeric = True)
new_sat_result['SAT Writing Avg. Score'] = new_sat_result['SAT Writing Avg. Score'].convert_objects(convert_numeric = True)

/Users/Derrick/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app
/Users/Derrick/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()
/Users/Derrick/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [8]:
new_sat_result["Total_Avg"] = new_sat_result.apply(lambda x: x['SAT Math Avg. Score'] + x['SAT Critical Reading Avg. Score'] + x['SAT Writing Avg. Score'], axis=1)

In [11]:
new_sat_result = new_sat_result.rename(columns={"Total_Avg":"SAT_Avg",})
sat_results_final = new_sat_result[['DBN', 'SAT_Avg']]